In [ ]:
# imports
%matplotlib inline   
                     # this sets up matplotlib to make plots show up in the notebook
import numpy as np   # imports the numpy package, abbreviated as np
import matplotlib    # imports the matplotlib package for making plots
import matplotlib.pyplot as plt    # imports the part of matplotlib we use most,
 
import numpy.random as random
import scipy.stats as stats


# Reviewing some things from before . . .

Set up two sets of 18 values from normal distributions, N(0,1) or N(1,1):

In [ ]:
#initialize the arrays
ndata=18
data1=random.randn(ndata)
data2=random.randn(ndata)+1.

In [ ]:
#find mean of each array
mean1=np.mean(data1)
mean2=np.mean(data2)

sigma1=np.std(data1,ddof=1)/np.sqrt(ndata) # want the standard deviation of the mean of data1
sigma2=np.std(data2,ddof=1)/np.sqrt(ndata) # want the standard deviation of the mean of data2


print(f'means: {mean1:.4f} , {mean2:.4f}')
print(f'sigmas: {sigma1:.4f} , {sigma2:.4f}')

tfactor=stats.t.ppf(1-0.025,ndata-1)
print(f'Confidence Interval for mean 1: [ {mean1-tfactor*sigma1:.4f} , {mean1+tfactor*sigma1:.4f} ]')
print(f'Confidence Interval for mean 2: [ {mean2-tfactor*sigma2:.4f} , {mean2+tfactor*sigma2:.4f} ]')


Confidence intervals for the difference:

In [ ]:
mean_diff=mean2-mean1
sigma_diff=np.sqrt(sigma1**2 + sigma2**2)
tfactor=stats.t.ppf(1-0.025, 2*ndata-2)

print(f'Observed difference of means: {mean_diff:.4f} ')

print(f'2-sided Confidence Interval: [ {mean_diff-tfactor*sigma_diff:.4f} , {mean_diff+tfactor*sigma_diff:.4f} ]')

tfactor=stats.t.ppf(1-0.05, 2*ndata-2)
print(f'1-sided Confidence Interval: > {mean_diff-tfactor*sigma_diff:.4f}')

# Permutation tests

We will combine `data1` and `data2` into 1 array; then generate sets of 2 datasets of size `ndata` and see how often their means differ as much as in the observed case.

In [ ]:
# choose number of bootstrap samples
nsims=int(5E4)

# make a combined dataset from both original data arrays
datac=np.concatenate( (data1,data2) )

# generate the two bootstrap samples
fake1=np.random.choice(datac,size=(ndata,nsims) )
fake2=np.random.choice(datac,size=(ndata,nsims) )



## Application to means

In [ ]:
# Calculate the means for each simulated dataset: should be nsims elements in each array
fakemeans1 = np.mean(fake1,axis=0)
fakemeans2 = np.mean(fake2,axis=0)

# Calculate the difference between the means
diffs=fakemeans2-fakemeans1  

__Using the below code box, plot histograms of the distributions of `fakemeans1` and `fakemeans2`, using the same binning and ~100 bins.__



__Using the below code box, plot a histogram of the distribution of differences between the means of each sample(`diffs`), using ~100 bins.  Add a vertical dashed line at the observed value of the difference between the means of the two real data samples (`data1` and `data2`) (you can use `plt.axvline` for this).__

In [ ]:
# print significance (alpha) = 0.32, 0.05, 0.01, and 0.001 limits on mean2-mean1
print(f'cutoffs: {np.percentile(diffs,(68.,95.,99.,99.9))}')

#make some empty space
print()

# compare the observed difference between the means to these cutoffs
print(f'Observed difference of means: {mean_diff:.4f} ')
print()

We can get a p-value using `scipy.stats.percentileofscore(array,value)`, which returns the percentile in `array` corresponding to the observed value `value`.

In [ ]:
print(f'p-value: {( 100-stats.percentileofscore(diffs,mean2-mean1) )/100.:.6g}')

## Application to standard deviations

We can also use the permutation test to investigate whether the ratio of standard deviations of the two samples could be different (note that each array by construction had the same intrinsic standard deviation, 1).

In [ ]:
# calculate std. dev. of each bootstrap fake sample

fakesigmas1=np.std(fake1,axis=0,ddof=1)
fakesigmas2=np.std(fake2,axis=0,ddof=1)

ratio=fakesigmas2/fakesigmas1
realratio=np.std(data2,ddof=1)/np.std(data1,ddof=1)

__Using the below code box, plot histograms of the two `fakesigmas` arrays, using ~100 bins each and the same binning.__

__Using the below code box, plot a histogram of the ratio of the two sigmas, i.e. `ratio=fakesigma2/fakesigma1`, with ~100 bins.  Put a vertical line at the observed value of the ratio calculated from `data1` and `data2`.__

__Using the below code box, determine the limits of the 68 and 95 percent regions (NOT the 68/95th percentile points, as we want a 2-sided test) for the value of this ratio, determined using the permutation tests.__ 

__Compare these limits to the observed value.__

__Also calculate the p-value for the observed ratio (again, keeping in mind that we want to do a 2-sided test, so we want to calculate the probability of anything either more extremely large OR more extremely small under the hypothesis that there is no difference): so this must be twice as large as the difference of your observed percentile/100  and either 0 or 1, whichever is closer...__

__What do you conclude about whether the observed ratio of standard deviations is consistent with the hypothesis of no difference?__

# The Kolmogorov-Smirnov Test

Let's set up two sets of Poisson-distributed data, with mean 5 or 7.5, and see if we can detect differences.  

In [ ]:
ndata = 25
countb = stats.poisson.rvs(5,size=(ndata) )
countr =stats.poisson.rvs(7.5,size=(ndata) )

__Use the below code box to bring up the help on `stats.ks_2samp`.__

__Evaluate whether the K-S test below finds a statistically significant difference between the two samples.__

In [ ]:
d,p = stats.ks_2samp(countb,countr)
print(f'd value: {d:.4f} , p-value: {p:.6g}')

# The Mann-Whitney U Test

__Use the below code box to bring up the help on `stats.mannwhitneyu`.__

__Evaluate whether the U test below finds a statistically significant difference between the two samples.__

In [ ]:
u,p = stats.mannwhitneyu(countb,countr)
print(f'U value: {u:.4f} , p-value: {p:.4g}')
